# Definição da Classe para Gerenciar a tela

In [1]:
class Screen(object):

    # Construtor da classe
    def __init__(self, title, bgColor, width, height):
        self.title = title       # título da janela
        self.bgColor = bgColor   # cor de fundo
        self.width = width       # largura da janela
        self.height = height     # altura da janela
        self.screen = pygame.display.set_mode(self.size()) # define o tamanho da tela
        pygame.display.set_caption(self.title)             # define o título da janela
        self.clock = pygame.time.Clock()
        
        
    # Executa o pipeline gráfico
    def run(self, obj):
        while True:  # laço principal
            # captura eventos
            for event in pygame.event.get(): 
                
                # Captura evento de clicar em botão para fechar
                if event.type == pygame.QUIT:
                    return pygame.quit()
            
            # preencha a tela com a cor de fundo
            self.screen.fill(self.bgColor)
            
            # gera o desenho
            obj.draw(self);
            
            # aplica o antialiasing
            # self.meanFilter()
            
            # atualiza a tela 
            pygame.display.update()
            
            self.clock.tick(30)
        
        
    # retorna um vetor com o tamanho da tela
    def size(self):
        return (self.width, self.height)
    
    # modifica um pixel na tela com a cor desejada
    def setPixel(self, x, y, color):
        self.screen.set_at((x, y), color)
    
    # filtro da média para o antialising
    def meanFilter(self):
        # Captura a matrix da tela
        #frameBuffer2 = pygame.PixelArray(self.screen)
        
        from copy import copy
        frameBuffer = pygame.surfarray.array3d(self.screen)
        #print(frameBuffer)
        
        import numpy as np
        mask = np.ones((3, 3)) * 1/9 
        
        #print(mask)
        temp = np.zeros((3))
               
        for i in range(1, self.width - 1):
            for j in range(1, self.height - 1):    
                subI = frameBuffer[i-1:i+2,j-1:j+2, 0]
                temp[0] = np.sum(np.multiply(subI, mask))
                #display(temp)
                
                subI = frameBuffer[i-1:i+2,j-1:j+2, 1]
                temp[1] = np.sum(np.multiply(subI, mask))
                #display(temp)
                
                subI = frameBuffer[i-1:i+2,j-1:j+2, 2]
                temp[2] = np.sum(np.multiply(subI, mask))
                #display(temp)
                
                #temp = np.zeros((3))
                
                #for k in range(-1,2):
                #    for l in range(-1,2):
                #        for b in range(3):
                #            temp[b] = temp[b] + frameBuffer[i + k][j + l][b] * mask[k + 1][l + 1]
                        
                #print(pygame.Color(int(temp[0]), int(temp[1]), int(temp[2]), 255))
                        
                self.setPixel(i, j, pygame.Color(int(temp[0]), int(temp[1]), int(temp[2]), 255));
                #frameBuffer2[i][j] = pygame.Color(int(temp[0]), int(temp[1]), int(temp[2]), 255);

# Definição da Classe para Primitiva de Linha

In [2]:
class Primitive(object):
    # construtor da classe
    def __init__(self, id):
        self.id = id
        

class Line(Primitive):
    # construtor da classe
    def __init__(self, x1, y1, x2, y2, color):
        self.x1 = x1       # coordenada x do primeiro ponto
        self.x2 = x2       # coordenada x do segundo ponto
        self.y1 = y1       # coordenada y do primeiro ponto
        self.y2 = y2       # coordenada y do segundo ponto
        self.color = color # cor do objeto
    
    # renderiza a linha desejada na tela
    def draw(self, screen):
        self.dda(screen)

    # Algoritmo DDA
    def dda(self, screen):      
        # Definição e Inicialização de Variáveis locais
        dx, dy, k = 0, 0, 0
        x_inc, y_inc = 0.0, 0.0
        x, y = 0.0, 0.0
    
        # Define os deslocamentos nas direções x e y
        dx = self.x2 - self.x1
        dy = self.y2 - self.y1
    
        # Define qual a direção de incremento fixo
        if abs(dx) > abs(dy):
            iter = abs(dx)
        else:
            iter = abs(dy)
        
        # Define os incrementos para cada direção
        x_inc = dx/iter
        y_inc = dy/iter

        # Define o ponto inicial
        x = self.x1
        y = self.y1

        # Desenha o ponto inicial na tela
        screen.setPixel(round(x), round(y), self.color)

        # Geração e renderização dos pontos seguintes da linha
        for k in range(iter):
            # Gera o próximo ponto
            x = x + x_inc
            y = y + y_inc
            
            # Desenha o ponto
            screen.setPixel(round(x), round(y), self.color)
            
    def bresenham(self, screen):
        # Definição e inicialização de variáveis locais
        dx, dy, d = 0, 0, 0
        incrE, incrNE = 0, 0
        x, y, xFinal = 0, 0, 0
        
        # Define os deslocamentos absolutos nas direções x e y
        dx = abs(self.x2 - self.x1)
        dy = abs(self.y2 - self.y1)
        
        # Define o d de teste inicial
        d = 2 * dy - dx
        
        # Define os incrementos nas direções x e y
        incrE = 2 * dy
        incrNE = 2 * (dy - dx)
        
        # Troca a ordem dos pontos em caso de segundo ponto à esquerda de primeiro ponto
        if self.x1 > self.x2:
            x = self.x2
            y = self.y2
            xFinal = self.x1
        else:
            x = self.x1
            y = self.y1
            xFinal = self.x2
        
        # Desenha o ponto inicial na tela
        screen.setPixel(x, y, self.color)
        
        # Gera e renderiza os pontos seguintes da linha
        while x < xFinal:
            # Gera o próximo ponto
            x = x + 1
            
            if d < 0:
                d = d + incrE
            else:
                y = y + 1
                d = d + incrNE
            
            # Desenha o próximo ponto
            screen.setPixel(x, y, self.color)
    
    def getListOfPoints(self):
        l = []
        l.append(Point3D(self.x1, self.y1, 0))
        l.append(Point3D(self.x2, self.y2, 0))
        return l
    
    def transform(self,T):

        point = np.array([self.x1,self.y1,0,1]).T
        res1 = T.matrix.dot(point)
        

        point = np.array([self.x2,self.y2,0,1]).T
        res2 = T.matrix.dot(point)
        
        l = Line(int(round(res1[0])),int(round(res1[1])),int(round(res2[0])),int(round(res2[1])),self.color)

        return l


    # def transform(self, T):
    #     p = Polygon(self.showEdges, self.edgeColor, self.isFilled, self.fillColor)
    #     for item in self.listOfPoints:
    #         point = np.array([item.x, item.y, 0, 1]).T
    #         res = T.matrix.dot(point)
    #         p.addVertex(Point2D(int(round(res[0])), int(round(res[1])), item.color))
        
    #     return p


# Programa Principal

In [3]:
# Carregamento de bibliotecas
import pygame

pygame 2.1.2 (SDL 2.0.16, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
# Inicialização do PyGame
pygame.init()            

(5, 0)

In [5]:
# Criação do Objeto de Tela
# screen = Screen("Tela", pygame.Color(255, 255, 255, 255), 700, 700)

In [6]:
# Definição de uma linha de coordenadas (10,10) a (100,100) com cor preta
# line1 = Line(10, 10, 100, 100, pygame.Color(0, 0, 0, 255))

In [7]:
# Execução do Programa
# screen.run(line1)

# Definição da Classe para Primitiva de Circunferência

In [8]:
class Circle(Primitive):
    # construtor da classe
    def __init__(self, xc, yc, raio, color):
        self.xc = xc       # coordenada x do centro
        self.yc = yc       # coordenada y do centro
        self.raio = raio   # raio da circunferência
        self.color = color # cor do objeto
    
    # renderiza a circunferência desejada na tela
    # Algoritmo de Bresenham
    def draw(self, screen):
        # Definição e Inicialização de Variáveis locais      
        # Coniderando a circunferência ao redor da origem, mas renderizada transladada
        x = 0                # x inicial
        y = self.raio        # y inicial
        d = 1 - self.raio    # d de teste inicial

        # Desenha os pontos inicias de cada quadrante
        self.drawCirclePoints(x, y, screen)
        
        # Gera os novos pontos e os renderiza
        while x < y:
            
            if d < 0:   # Direção E
                d = d + 2 * x + 3
            else:       # Direção SE
                d = d + 2 * (x - y) + 5
                y = y - 1
            
            x = x + 1

            self.drawCirclePoints(x, y, screen)
            
    def drawCirclePoints(self, x, y, screen):
        xCentro = self.xc
        yCentro = self.yc
        screen.setPixel(xCentro + x, yCentro + y, self.color)
        screen.setPixel(xCentro + y, yCentro + x, self.color)
        screen.setPixel(xCentro + y, yCentro - x, self.color)
        screen.setPixel(xCentro + x, yCentro - y, self.color)
        screen.setPixel(xCentro - x, yCentro - y, self.color)
        screen.setPixel(xCentro - y, yCentro - x, self.color)
        screen.setPixel(xCentro - y, yCentro + x, self.color)
        screen.setPixel(xCentro - x, yCentro + y, self.color)
    
    def transladar(self, Tx,Ty):
        c = Circle(self.xc + Tx, self.yc + Ty, self.raio, self.color)
        
        return c

# Criação de uma classe que contém o desenho

In [9]:
# class Picture(object):
#     # Construtor da Classe
#     def __init__(self):
#         self.primitivas = [] # Define uma lista de primitivas para representar um desenho
    
#     def draw(self, screen):
#         # Telhado
#         l1 = Line(350, 50, 50, 250, pygame.Color(255, 0, 0, 255))
#         l2 = Line(50, 250, 650, 250, pygame.Color(255, 0, 0, 255))
#         l3 = Line(650, 250, 350, 50, pygame.Color(255, 0, 0, 255))
        
#         # Parede
#         l4 = Line(50, 250, 50, 650, pygame.Color(0, 200, 100, 255))
#         l5 = Line(50, 650, 650, 650, pygame.Color(0, 200, 100, 255))
#         l6 = Line(650, 650, 650, 250, pygame.Color(0, 200, 100, 255))
        
#         circ = Circle(350, 450, 150, pygame.Color(0, 0, 255, 255));
        
#         # Insere primitivas na lista
#         self.primitivas.append(l1);
#         self.primitivas.append(l2);
#         self.primitivas.append(l3);
#         self.primitivas.append(l4);
#         self.primitivas.append(l5);
#         self.primitivas.append(l6);
#         self.primitivas.append(circ);
        
#         # Desenha cada primitiva que está na lista
#         for item in self.primitivas:
#             item.draw(screen);
        

In [10]:
# Lembre-se de carregar novamente a tela
# Criação do Objeto de Tela
# screen = Screen("Tela", pygame.Color(255, 255, 255, 255), 700, 700)

# # executa o desenho
# circ = Circle(350, 450, 150, pygame.Color(0, 0, 255, 255));

# screen.run(circ);

In [11]:
# Lembre-se de carregar novamente a tela
# Criação do Objeto de Tela
# screen = Screen("Tela", pygame.Color(255, 255, 255, 255), 700, 700)

# # executa o desenho
# pic = Picture();

# screen.run(pic);

# Classe da primitiva Ponto

In [12]:
class Point2D(object):
    # construtor da Classe
    def __init__(self, x, y, color):
        self.x = x
        self.y = y
        self.color = color
    
    # Renderiza um ponto
    def draw(self, screen):
        screen.setPixel(self.x, self.y, self.color)
        
    @property
    def x(self):
        return self._x
    
    @x.setter
    def x(self, x):
        self._x = x
    
    @property
    def y(self):
        return self._y
    
    @y.setter
    def y(self, y):
        self._y = y
        
    @property
    def color(self):
        return self._color
    
    @color.setter
    def color(self, color):
        self._color = color

# Classe para informação de Arestas

In [13]:
class EdgeInfo(object):
    # Construtor da Classe
    def __init__(self, initialPoint, finalPoint):
        if initialPoint.y <= finalPoint.y:
            self.yMax = finalPoint.y
            self.x = initialPoint.x  # x corrente, inicialmente x in Ymin
            self.yMin = initialPoint.y
        else:
            self.yMax = initialPoint.y
            self.x = finalPoint.x     # x corrente, inicialmente x in Ymin
            self.yMin = finalPoint.y
            
        self.inverseOfAngularCoefficient = (finalPoint.x - initialPoint.x) \
                                            / (finalPoint.y - initialPoint.y)
            
    @property
    def yMax(self):
        return self._yMax
    
    @yMax.setter
    def yMax(self, yMax):
        self._yMax = yMax
    
    @property
    def yMin(self):
        return self._yMin
    
    @yMin.setter
    def yMin(self, yMin):
        self._yMin = yMin
    
    @property
    def x(self):
        return self._x
    
    @x.setter
    def x(self, x):
        self._x = x
    
    def updateX(self):
        self.x = self.x + self.inverseOfAngularCoefficient

# Classe da primitiva Polígono

In [14]:
class Polygon(Primitive):
    # construtor da classe
    def __init__(self, showEdges, edgeColor, isFilled, fillColor):
        self.listOfPoints = []
        self.showEdges = showEdges
        self.edgeColor = edgeColor
        self.isFilled = isFilled
        self.fillColor = fillColor
    
    # Adiciona vértices na lista
    def addVertex(self, point):
        self.listOfPoints.append(point)
    
    # Renderiza Polígono
    def draw(self, screen):
        if len(self.listOfPoints) < 3:
            print("Não forma polígono. Menos de 3 vértices.")
            return
            
        # Desenha arestas se desejar
        if self.isFilled:
            self.scanline(screen)
        
        if self.showEdges:
            for i in range(0,len(self.listOfPoints) - 1):     
                pI = self.listOfPoints[i]
                pF = self.listOfPoints[i+1]
                line = Line(pI.x, pI.y, pF.x, pF.y, self.edgeColor)
                line.draw(screen)
            
            pI = self.listOfPoints[-1]
            pF = self.listOfPoints[0]
            line = Line(pI.x, pI.y, pF.x, pF.y, self.edgeColor)
            line.draw(screen)
            
            for i in range(0,len(self.listOfPoints)):               
                self.listOfPoints[i].draw(screen)
    
    # Faz o scanline para preencher o polígono
    def scanline(self, screen): 
        yMax = self.listOfPoints[0].y
        for item in self.listOfPoints:
            if item.y > yMax:
                yMax = item.y
                
        y = yMax # armazena o y corrente, começando pelo valor mínimo
        
        #### Cria tabela de arestas ####
        edgeTable = []
        for i in range (0, yMax+1):
            edgeTable.append([])
            
        for i in range(0,len(self.listOfPoints) - 1):     
        #for i in range(0,1):     
            # exclui arestas horizontais
            if self.listOfPoints[i].y - self.listOfPoints[i+1].y != 0:
                edge = EdgeInfo(self.listOfPoints[i], self.listOfPoints[i+1])
                yMin = edge.yMin
                if yMin < y:
                    y = yMin
    
                edgeTable[yMin].append(edge)       
        
        # Fecha o polígono
        # exclui arestas horizontais
        if self.listOfPoints[-1].y - self.listOfPoints[0].y != 0:
            edge = EdgeInfo(self.listOfPoints[-1], self.listOfPoints[0])
            yMin = edge.yMin
            if yMin < y:
                y = yMin
            edgeTable[yMin].append(edge)
      
        ####
        activeET = []
        
        ### Laço principal
        while y <= yMax:
            #print(y)
                    
            # Move a lista y na ET para AET (ymin = y), mantendo a AET ordenada em x
            activeET.extend(edgeTable[y])
            edgeTable[y] = []
            activeET.sort(key = sortByX)
            
            # Desenhe os pixels do bloco na linha de varredura y, 
            # usando os pares de coordenadas x da AET (cada dois nós definem um bloco)
            for i in range(0, len(activeET) - 1, 2):
                for x in range(int(activeET[i].x), int(activeET[i + 1].x + 1)):
                    screen.setPixel(x, y, self.fillColor)
            
            # Atualiza o valor de y para a próxima linha de varredura
            y = y + 1
            
            # Remova as arestas que possuem ymax = y da AET
            delL = []
            for item in activeET:
                if item.yMax <= y:
                    delL.append(item)
            
            for item in delL:
                activeET.remove(item)
            
            delL.clear()
            
            # Para cada aresta na AET, atualize x = x + 1/m
            for item in activeET:
                item.updateX()
                
                
    def empty(self, ET):
        for item in ET:
            if item:
                return False
        
        return True
    
    def getListOfPoints(self):
        return self.listOfPoints
    
    def transform(self, T):
        p = Polygon(self.showEdges, self.edgeColor, self.isFilled, self.fillColor)
        for item in self.listOfPoints:
            point = np.array([item.x, item.y, 0, 1]).T
            res = T.matrix.dot(point)
            p.addVertex(Point2D(int(round(res[0])), int(round(res[1])), item.color))
        
        return p

# Usada para ordenar a AET por valores de x
# Observe que a função não pertence à classe Polygon
def sortByX(item):
    return item.x
    

In [15]:
# screen = Screen("Tela", pygame.Color(255, 255, 255, 255), 700, 700)

# pol = Polygon(True, pygame.Color(255, 0, 0, 255), True, pygame.Color(255, 255, 0, 255))
# pol.addVertex(Point2D(20, 30, pygame.Color(0, 0, 0, 255)))
# pol.addVertex(Point2D(70, 10, pygame.Color(0, 0, 0, 255)))
# pol.addVertex(Point2D(130, 50, pygame.Color(0, 0, 0, 255)))
# pol.addVertex(Point2D(130, 100, pygame.Color(0, 0, 0, 255)))
# pol.addVertex(Point2D(70, 70, pygame.Color(0, 0, 0, 255)))
# pol.addVertex(Point2D(20, 90, pygame.Color(0, 0, 0, 255)))

# # Execução do Programa
# screen.run(pol)

In [16]:
import numpy as np
import math

class GeometricTransformation(object):
    # construtor da Classe
    def __init__(self, matrix):
        self.matrix = matrix

    @property
    def matrix(self):
        return self._matrix
    
    @matrix.setter
    def matrix(self, matrix):
        self._matrix = matrix


class Translation3D(GeometricTransformation):
    # construtor da classe
    def __init__(self, Tx, Ty, Tz):
        super().__init__(np.array([[1, 0, 0, Tx], [0, 1, 0, Ty], [0, 0, 1, Tz], [0, 0, 0, 1]]))
    
class Scale3D(GeometricTransformation):
    # construtor da classe
    def __init__(self, Sx, Sy, Sz):
        super().__init__(np.array([[Sx, 0, 0, 0], [0, Sy, 0, 0], [0, 0, Sz, 0], [0, 0, 0, 1]]))
        
class Rotation3D(GeometricTransformation):
    # construtor da classe
    def __init__(self, theta, axis):
        self.axis = axis
        
        if axis == 'z':
            super().__init__(np.array([[math.cos(theta), -math.sin(theta), 0, 0], [math.sin(theta), math.cos(theta), 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]))
        elif axis == 'x':
            super().__init__(np.array([[1, 0, 0, 0], [0, math.cos(theta), -math.sin(theta), 0], [0, math.sin(theta), math.cos(theta), 0], [0, 0, 0, 1]]))
        elif axis == 'y':
            super().__init__(np.array([[math.cos(theta), 0, math.sin(theta), 0], [0, 1, 0, 0], [-math.sin(theta), 0, math.cos(theta), 0], [0, 0, 0, 1]]))
        else:
            print("Eixo não definido!")
            exit()

class CombinedTransformation(GeometricTransformation):
    # construtor da classe
    def __init__(self):
        super().__init__([])
        self.stack = []
    
    def add(self, matrix):
        self.stack.append(matrix)
    
    #def pop(self):
    #    return self.stack.pop(-1)
              
    def combine(self):
        while (len(self.stack) > 1):
            m2 = self.stack.pop(-1)
            m1 = self.stack.pop(-1)
            m3 = GeometricTransformation(m2.matrix.dot(m1.matrix))
            self.stack.append(m3)
            
        # Retira o item restante da pilha    
        # Esta é a matrix combinada
        self.matrix = self.stack.pop(0).matrix
              
class Point3D(Point2D):
    # construtor da Classe
    def __init__(self, x, y, z, color):
        super().__init__(x, y, color)
        self.z = z
            
    @property
    def z(self):
        return self._z
    
    @z.setter
    def z(self, z):
        self._z = z
        
class Axis3D(object):
    # construtor da Classe
    def __init__(self, x, y, z):
        self.v = np.array([x, y, z])
    
    def norm(self):
        return (self.v**2).sum()**0.5
        
    def getUnityVector(self):
        temp = self.v / self.norm()
        
        return Axis3D(temp[0], temp[1], temp[2])
    
    @property
    def v(self):
        return self._v
    
    @v.setter
    def v(self, v):
        self._v = v
                      
class RotationInArbitraryAxis(CombinedTransformation):
    # construtor da classe
    def __init__(self, pointA, pointB, theta):
        super().__init__()
        super().add(Translation3D(-pointA.x, -pointA.y, -pointA.z))
        V = Axis3D(pointB.x - pointA.x, pointB.y - pointA.y, pointB.z - pointA.z)
        u = V.getUnityVector()
        d = np.sqrt(u.v[1]**2 + u.v[2]**2)
        alpha = np.arccos(u.v[2] / d)
        super().add(Rotation3D(alpha, 'x'))
        beta = np.arccos(d)
        super().add(Rotation3D(beta, 'y'))
        super().add(Rotation3D(theta, 'z'))
        super().add(Rotation3D(-beta, 'y'))
        super().add(Rotation3D(-alpha, 'x'))
        super().add(Translation3D(pointA.x, pointA.y, pointA.z))
        super().combine()
        
class ScaleWithFixedPoint(CombinedTransformation):
    # construtor da classe
    def __init__(self, point, Sx, Sy, Sz):
        super().__init__()
        super().add(Translation3D(-point.x, -point.y, -point.z))
        super().add(Scale3D(Sx, Sy, Sz))
        super().add(Translation3D(point.x, point.y, point.z))
        super().combine()
        
class WindowToViewportTransformation(CombinedTransformation):
    # construtor da classe
    def __init__(self, xd1, yd1, xd2, yd2, xv1, yv1, xv2, yv2):
        super().__init__()
        super().add(Translation3D(-xd1, -yd1, 0))
        x_view_factor = (xv2 - xv1) / (xd2 - xd1)
        y_view_factor = (yv2 - yv1) / (yd2 - yd1)
        super().add(Scale3D(x_view_factor, y_view_factor, 0))
        super().add(Translation3D(xv1, yv1, 0))
        super().combine()

In [17]:
class Perna(Primitive):
    def __init__(self,coxa,canela,pe):
        self.coxa = coxa
        self.canela = canela
        self.pe = pe

        self.ponto1coxa = Point3D(self.coxa.x1,self.coxa.y1 , 0, pygame.Color(255, 0, 0, 255))
        self.ponto2coxa = Point3D(self.coxa.x1,self.coxa.y1, 1, pygame.Color(255, 0, 0, 255))

        self.ponto1canela = Point3D(self.canela.x1,self.canela.y1 ,0 , pygame.Color(255, 0, 0, 255))
        self.ponto2canela = Point3D(self.canela.x1,self.canela.y1,1, pygame.Color(255, 0, 0, 255))

        self.ponto1pe = Point3D(self.pe.x1, self.pe.y1,0 , pygame.Color(255, 0, 0, 255))
        self.ponto2pe = Point3D(self.pe.x1, self.pe.y1,1 , pygame.Color(255, 0, 0, 255))

        # Deve-se atualizar esses pontos acima

    def updatePontos(self):
        self.ponto1coxa = Point3D(self.coxa.x1,self.coxa.y1 , 0, pygame.Color(255, 0, 0, 255))
        self.ponto2coxa = Point3D(self.coxa.x1,self.coxa.y1, 1, pygame.Color(255, 0, 0, 255))

        self.ponto1canela = Point3D(self.canela.x1,self.canela.y1 ,0 , pygame.Color(255, 0, 0, 255))
        self.ponto2canela = Point3D(self.canela.x1,self.canela.y1,1, pygame.Color(255, 0, 0, 255))

        self.ponto1pe = Point3D(self.pe.x1, self.pe.y1,0 , pygame.Color(255, 0, 0, 255))
        self.ponto2pe = Point3D(self.pe.x1, self.pe.y1,1 , pygame.Color(255, 0, 0, 255))

    def draw(self, screen):
        self.coxa.draw(screen)
        self.canela.draw(screen)
        self.pe.draw(screen)
    
    def RotacionarCoxa(self,theta):
        T = RotationInArbitraryAxis(self.ponto1coxa,self.ponto2coxa,theta)
        Xantes = self.coxa.x2
        Yantes = self.coxa.y2

        coxa2 = self.coxa.transform(T)
        self.coxa = coxa2

        Xdepois = self.coxa.x2
        Ydepois = self.coxa.y2

        # Tx = (self.coxa.x2 * math.cos(theta) - self.coxa.y2 * math.sin(theta)) - self.coxa.x2 # CORRIGIR
        # Ty = (self.coxa.y2 * math.cos(theta) + self.coxa.x2 * math.sin(theta)) - self.coxa.y2 # CORRIGIR
        # Tz = 0

        T = Translation3D((Xdepois-Xantes), (Ydepois-Yantes), 0) # 
        canela2 = self.canela.transform(T)
        self.canela = canela2

        pe2 = self.pe.transform(T)
        self.pe = pe2

        self.updatePontos()

        return self

    def RotacionarCanela(self,theta):
        T = RotationInArbitraryAxis(self.ponto1canela,self.ponto2canela,theta)

        Xantes = self.canela.x2
        Yantes = self.canela.y2

        canela2 = self.canela.transform(T)
        self.canela = canela2


        Xdepois = self.canela.x2
        Ydepois = self.canela.y2

        T = Translation3D((Xdepois-Xantes), (Ydepois-Yantes), 0)
        pe2 = self.pe.transform(T)
        self.pe = pe2

        self.updatePontos()

        return self

    def RotacionarPe(self,theta):
        T = RotationInArbitraryAxis(self.ponto1pe,self.ponto2pe,theta)
        pe2 = self.pe.transform(T)
        self.pe = pe2

        self.updatePontos()

        return self
    
    

        

In [18]:
class MovimentarCoxa(CombinedTransformation):
    # construtor da classe
    def __init__(self, theta, x,y):
        super().__init__()
        # Rotacionar Coxa
        super().add(RotationInArbitraryAxis(x,y,theta))
        
        # Transladar perna o tanto que o segundo ponto da coxa transladou
        #super().add(Translation3D(x*math.cos,,0))
        # transladar pé o tanto que o segundo ponto da coxa transladou

        super().combine()

In [19]:
class Picture2(object):
    # Construtor da Classe
    def __init__(self):
        self.primitivas = [] # Define uma lista de primitivas para representar um desenho
        self.count = 0
        self.count2 = 0
        self.count3 = 0
        self.count4 = 0
        self.count5 = 0
        self.coxa = Line(200, 300, 400, 375, pygame.Color(255, 0, 0, 255))
        self.canela = Line(400, 375, 380, 575, pygame.Color(255, 0, 0, 255))
        self.pe = Line(380, 575, 475, 575, pygame.Color(255, 0, 0, 255))
        self.perna = Perna(self.coxa,self.canela,self.pe)
        self.bola = Circle(540,463,35,pygame.Color(255, 0, 0, 255))
    
    def draw(self, screen):

        screen = Screen("Tela", pygame.Color(255, 255, 255, 255), 700, 700)


        if self.count < 13: # movimento 1 da perna  
            self.count += 1
            self.perna.RotacionarCoxa(-math.pi * 2/180)
            
        elif self.count2 < 13: # movimento 2 da perna  
            self.count2 += 1
            self.perna.RotacionarCanela(math.pi * 2/180)
            #self.primitivas.append(self.perna)

        elif self.count3 < 25: # movimento 3 e 4 da perna  
            self.count3 += 1
            self.perna.RotacionarCanela(-math.pi * 1.8/180)
            self.perna.RotacionarPe(math.pi * 2/410)

        elif self.count4 < 16: # movimento da bola 
            self.count4 += 1
            self.bola = self.bola.transladar(8,-8)
        
        elif self.count5 < 60: # movimento da bola batendo na parede
            self.count5 += 1
            self.bola = self.bola.transladar(-7,-7)


        # Insere primitivas na lista
        self.primitivas.append(self.perna)
        self.primitivas.append(self.bola)
        
        

        
        
        # Desenha cada primitiva que está na lista
        for item in self.primitivas:
            item.draw(screen)
        
        self.primitivas.clear()

In [20]:
# Lembre-se de carregar novamente a tela
# Criação do Objeto de Tela
screen = Screen("Tela", pygame.Color(255, 255, 255, 255), 700, 700)

# executa o desenho
pic = Picture2();

screen.run(pic);